In [8]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
import faiss
from transformers import pipeline

In [9]:
df = pd.read_csv("Training Dataset.csv")
print("Total rows in dataset:", len(df))
df.head()

Total rows in dataset: 614


,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [10]:
documents = []

for _, row in df.iterrows():
    doc = f"""
    Applicant ID: {row['Loan_ID']}
    Gender: {row['Gender']}
    Married: {row['Married']}
    Education: {row['Education']}
    Self Employed: {row['Self_Employed']}
    Applicant Income: {row['ApplicantIncome']}
    Coapplicant Income: {row.get('CoapplicantIncome', 'N/A')}
    Loan Amount: {row['LoanAmount']}
    Loan Amount Term: {row.get('Loan_Amount_Term', 'N/A')}
    Credit History: {row.get('Credit_History', 'N/A')}
    Property Area: {row.get('Property_Area', 'N/A')}
    Loan Status: {row['Loan_Status']}
    """
    documents.append(doc.strip())

In [11]:
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
doc_embeddings = embedding_model.encode(documents)

index = faiss.IndexFlatL2(doc_embeddings.shape[1])
index.add(np.array(doc_embeddings))

print("Documents embedded and index created.")

Documents embedded and index created.


In [ ]:
generator = pipeline("text2text-generation", model="google/flan-t5-small")

In [13]:
def retrieve_top_k(query, k=3):
    query_embedding = embedding_model.encode([query])
    distances, indices = index.search(np.array(query_embedding), k)
    return [documents[i] for i in indices[0]]

def generate_answer(query, context_docs):
    context = "\n".join(context_docs)
    prompt = f"Answer the question based on the context below:\n{context}\n\nQuestion: {query}"
    result = generator(prompt, max_length=256, do_sample=False)
    return result[0]['generated_text']

In [14]:
while True:
    query = input("\nAsk a question about loan data (or type 'exit' to quit): ")
    if query.lower() == 'exit':
        print("Chatbot session ended.")
        break

    top_docs = retrieve_top_k(query)
    answer = generate_answer(query, top_docs)

    print("\nAnswer:")
    print(answer)

    print("\nContext Used:")
    for doc in top_docs:
        print("-----")
        print(doc)


Ask a question about loan data (or type 'exit' to quit):  How many applicants are self-employed?


Both `max_new_tokens` (=256) and `max_length`(=256) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



Answer:
3

Context Used:
-----
Applicant ID: LP002868
    Gender: Male
    Married: Yes
    Education: Graduate
    Self Employed: No
    Applicant Income: 3159
    Coapplicant Income: 461.0
    Loan Amount: 108.0
    Loan Amount Term: 84.0
    Credit History: 1.0
    Property Area: Urban
    Loan Status: Y
-----
Applicant ID: LP002983
    Gender: Male
    Married: Yes
    Education: Graduate
    Self Employed: No
    Applicant Income: 8072
    Coapplicant Income: 240.0
    Loan Amount: 253.0
    Loan Amount Term: 360.0
    Credit History: 1.0
    Property Area: Urban
    Loan Status: Y
-----
Applicant ID: LP002277
    Gender: Female
    Married: No
    Education: Graduate
    Self Employed: No
    Applicant Income: 3180
    Coapplicant Income: 0.0
    Loan Amount: 71.0
    Loan Amount Term: 360.0
    Credit History: 0.0
    Property Area: Urban
    Loan Status: N



Ask a question about loan data (or type 'exit' to quit):  Tell me about the loan status of female applicants.


Both `max_new_tokens` (=256) and `max_length`(=256) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


...


Ask a question about loan data (or type 'exit' to quit):  What documents are required to apply for a loan?


Both `max_new_tokens` (=256) and `max_length`(=256) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



Answer:
Applicant ID: LP001228 Gender: Male Married: No Education: Not Graduate Self Employed: No Applicant Income: 3200 Coapplicant Income: 2254.0 Loan Amount Term: 180.0 Credit History: 0.0 Property Area: Urban Loan Status: N Applicant ID: LP001228 Gender: Male Married: No Education: Not Graduate Self Employed: No Applicant Income: 3200 Coapplicant Income: 2254.0 Loan Amount Term: 180.0 Credit History: 0.0 Property Area: Urban Loan Status: N

Context Used:
-----
Applicant ID: LP001994
    Gender: Female
    Married: No
    Education: Graduate
    Self Employed: No
    Applicant Income: 2400
    Coapplicant Income: 1863.0
    Loan Amount: 104.0
    Loan Amount Term: 360.0
    Credit History: 0.0
    Property Area: Urban
    Loan Status: N
-----
Applicant ID: LP001800
    Gender: Male
    Married: Yes
    Education: Not Graduate
    Self Employed: No
    Applicant Income: 2510
    Coapplicant Income: 1983.0
    Loan Amount: 140.0
    Loan Amount Term: 180.0
    Credit History: 1.0
   


Ask a question about loan data (or type 'exit' to quit):  exit


Chatbot session ended.
